In [200]:
import collections
import http.client
import pandas as pd
from operator import itemgetter
import json
from collections import namedtuple
pd.set_option("display.max_rows", 200)
pd.set_option('display.max_colwidth', -1)

In [201]:
Rounds = namedtuple('Rounds', 'player round score strokes thru')

In [179]:
conn = http.client.HTTPConnection("api.sportradar.us")
conn.request("GET", "/golf-t2/leaderboard/pga/2018/tournaments/b404a8d5-5e33-4417-ae20-5d4d147042ee/leaderboard.json?api_key=u2rg6jhkezj6dkutjcmfqc8e")
res = conn.getresponse()
data = res.read()
d = data.decode("utf-8")

In [180]:
missed_cut = ["Sandy Lyle", "Charl Schwartzel", "Jose Maria Olazabal", "Jason Dufner", "Patrick Cantlay", 
              "Shubhankar Sharma", "Thomas Pieters", "Ted Potter", "Brendan Steele", "Dylan Frittelli", 
              "Danny Willett", "Ross Fisher", "Wesley Bryan", "Pat Perez", "Patton Kizzire", "Trevor Immelman", 
              "Joaquin Niemann", "Yuta Ikeda", "Alexander Noren", "Kevin Chappell", "Doc Redman", "Gary Woodland", 
              "Yusaku Miyazato", "Austin Cook", "Billy Horschel", "Ian Woosnam", "Mike Weir", "Larry Mize", 
              "Angel Cabrera", "Yuxin Lin", "Mark O'Meara", "Matt Parziale", "Sergio Garcia", "Harry Ellis"]
missed_cut_players = [p.lower().replace(" ", "") for p in missed_cut]
# sorted(missed_cut_players)

In [199]:
cols = ["first_name", "last_name", "position", "score", "strokes", "full_name"]
lb_df = pd.DataFrame(json.loads(d)['leaderboard'])
lb_df["full_name"] = lb_df["first_name"].str.lower().str.replace(" ", "") + lb_df["last_name"].str.lower().str.replace(" ", "")
lb_df[cols]
lb_df.index += 1
lb_df.to_csv("leaderboard.csv", columns=cols)
lb_df[cols]

,first_name,last_name,position,score,strokes,full_name
1,Patrick,Reed,1,-14,202,patrickreed
2,Rory,McIlroy,2,-11,205,rorymcilroy
3,Rickie,Fowler,3,-9,207,rickiefowler
4,Jon,Rahm,4,-8,208,jonrahm
5,Henrik,Stenson,5,-7,209,henrikstenson
6,Marc,Leishman,6,-6,210,marcleishman
7,Tommy,Fleetwood,6,-6,210,tommyfleetwood
8,Bubba,Watson,6,-6,210,bubbawatson
9,Jordan,Spieth,9,-5,211,jordanspieth
10,Justin,Thomas,9,-5,211,justinthomas


In [196]:
sorted(list(lb_df[lb_df["status"] == 'CUT'].full_name)) == sorted(missed_cut_players)

True

In [182]:
def get_high_round(rounds):
    high_player = max(rounds, key=lambda x: x.strokes)
    return high_player
    
def get_score(player):
    player = player.lower().replace(" ", "")
    score = lb_df.loc[lb_df["full_name"] == player]["score"]
    strokes = lb_df.loc[lb_df["full_name"] == player]["strokes"]
    high_round = get_high_round(active_rounds)
    if player in missed_cut_players:
        return "**MC** Score: {} Strokes: {}".format(int(score) + high_round.score, int(strokes) + high_round.strokes)
    return "Score: {}, Strokes: {}".format(int(score), int(strokes))
    
def get_strokes(player):
    player = player.lower().replace(" ", "")
    high_strokes = 0
    if player in missed_cut_players:
        return int(lb_df.loc[lb_df["full_name"] == player]["strokes"]) + get_high_round(active_rounds).strokes
    else:
        return int(lb_df.loc[lb_df["full_name"] == player]["strokes"])

In [183]:
players_and_rounds = [("{} {}".format(r["first_name"], r["last_name"]), 
                       r["rounds"][2]) for r in json.loads(d)["leaderboard"] if len(r["rounds"]) > 2]
active_rounds = [Rounds(p[0], p[1]["sequence"], p[1]["score"], p[1]["strokes"], p[1]["thru"]) for p in players_and_rounds]

In [184]:
# sorted(active_rounds, key=lambda x: x.strokes, reverse=True)

In [185]:
df = pd.read_csv("masters.csv")
df = df[["Pool Player", "Player1", "Player2", "Player3", "Player4", 
         "Player5", "Player6", "Player7", "Total", "Tie Breaker"]]

In [186]:
for index, row in df.iterrows():
    score_accum = 0
    for player in row.values[1:8]:
        score_accum += get_strokes(player)
    df.set_value(index, "Total", score_accum)

In [187]:
df = df.sort_values(by=['Total']).reset_index(drop=True)
for i in range(1,8):
    df["Score{}".format(i)] = df["Player{}".format(i)].apply(lambda x: get_score(x))

In [188]:
high_round = get_high_round(active_rounds)
print("High round of the day belongs to {}, with score: {}, strokes: {}, thru: {}".format(
    high_round.player, high_round.score, high_round.strokes, high_round.thru))

High round of the day belongs to Vijay Singh, with score: 7, strokes: 79, thru: 18


In [190]:
df.index += 1
df = df[["Pool Player", 
         "Player1", "Score1",
         "Player2", "Score2",
         "Player3", "Score3",
         "Player4", "Score4",
         "Player5", "Score5",
         "Player6", "Score6",
         "Player7", "Score7",
         "Total", "Tie Breaker"]]
df

,Pool Player,Player1,Score1,Player2,Score2,Player3,Score3,Player4,Score4,Player5,Score5,Player6,Score6,Player7,Score7,Total,Tie Breaker
2,Paul Bower,Dustin Johnson,"Score: -4, Strokes: 212",Jason Day,"Score: -1, Strokes: 215",Bubba Watson,"Score: -6, Strokes: 210",Daniel Berger,"Score: 2, Strokes: 218",Adam Hadwin,"Score: 0, Strokes: 216",Tiger Woods,"Score: 4, Strokes: 220",Jimmy Walker,"Score: -1, Strokes: 215",1506.0,268.0
3,Dave Peterson,Rory McIlroy,"Score: -11, Strokes: 205",Jason Day,"Score: -1, Strokes: 215",Patrick Reed,"Score: -14, Strokes: 202",Tony Finau,"Score: -1, Strokes: 215",Jason Dufner,**MC** Score: 13 Strokes: 229,Tiger Woods,"Score: 4, Strokes: 220",Martin Kaymer,"Score: 5, Strokes: 221",1507.0,278.0
4,Sam Montgomery,Jordan Spieth,"Score: -5, Strokes: 211",Tommy Fleetwood,"Score: -6, Strokes: 210",Charley Hoffman,"Score: -1, Strokes: 215",Branden Grace,"Score: 4, Strokes: 220",Adam Hadwin,"Score: 0, Strokes: 216",Jimmy Walker,"Score: -1, Strokes: 215",Bryson DeChambeau,"Score: 4, Strokes: 220",1507.0,282.0
5,Joe Kearney,Justin Thomas,"Score: -5, Strokes: 211",Jason Day,"Score: -1, Strokes: 215",Patrick Reed,"Score: -14, Strokes: 202",Louis Oosthuizen,"Score: -3, Strokes: 213",Thomas Pieters,**MC** Score: 14 Strokes: 230,Tiger Woods,"Score: 4, Strokes: 220",Russell Henley,"Score: 0, Strokes: 216",1507.0,279.0
6,Zachary Sight,Justin Thomas,"Score: -5, Strokes: 211",Phil Mickelson,"Score: 7, Strokes: 223",Patrick Reed,"Score: -14, Strokes: 202",Daniel Berger,"Score: 2, Strokes: 218",Siwoo Kim,"Score: 0, Strokes: 216",Tiger Woods,"Score: 4, Strokes: 220",Fred Couples,"Score: 3, Strokes: 219",1509.0,278.0
7,Jeremy Szyba,Rickie Fowler,"Score: -9, Strokes: 207",Paul Casey,"Score: 2, Strokes: 218",Bubba Watson,"Score: -6, Strokes: 210",Louis Oosthuizen,"Score: -3, Strokes: 213",Jason Dufner,**MC** Score: 13 Strokes: 229,Jimmy Walker,"Score: -1, Strokes: 215",Zach Johnson,"Score: 2, Strokes: 218",1510.0,277.0
8,Tristen Woods,Jon Rahm,"Score: -8, Strokes: 208",Henrik Stenson,"Score: -7, Strokes: 209",Patrick Reed,"Score: -14, Strokes: 202",Brendan Steele,**MC** Score: 14 Strokes: 230,Cameron Smith,"Score: -3, Strokes: 213",Billy Horschel,**MC** Score: 18 Strokes: 234,Adam Scott,"Score: 2, Strokes: 218",1514.0,279.0
9,Jeremy Szyba #3,Dustin Johnson,"Score: -4, Strokes: 212",Phil Mickelson,"Score: 7, Strokes: 223",Patrick Reed,"Score: -14, Strokes: 202",Branden Grace,"Score: 4, Strokes: 220",Adam Hadwin,"Score: 0, Strokes: 216",Daniel Berger,"Score: 2, Strokes: 218",Ian Poulter,"Score: 7, Strokes: 223",1514.0,278.0
10,Chris Stine,Rory McIlroy,"Score: -11, Strokes: 205",Phil Mickelson,"Score: 7, Strokes: 223",Brian Harman,"Score: 7, Strokes: 223",Louis Oosthuizen,"Score: -3, Strokes: 213",Cameron Smith,"Score: -3, Strokes: 213",Tiger Woods,"Score: 4, Strokes: 220",Ryan Moore,"Score: 2, Strokes: 218",1515.0,280.0
11,Pat McNamara,Justin Thomas,"Score: -5, Strokes: 211",Phil Mickelson,"Score: 7, Strokes: 223",Bubba Watson,"Score: -6, Strokes: 210",Daniel Berger,"Score: 2, Strokes: 218",Cameron Smith,"Score: -3, Strokes: 213",Tiger Woods,"Score: 4, Strokes: 220",Bryson DeChambeau,"Score: 4, Strokes: 220",1515.0,276.0


In [163]:
df.to_csv("masters_results.csv")

# Tabula Fun

In [80]:
from tabula import read_pdf

In [81]:
df = read_pdf("/Users/jeremyszyba/Desktop/2018 Masters.pdf")
df = df.drop(df.index[10])
df

,Group 1,Group 2,Group 3,Group 4,Group 5,Field,Field.1,Field.2,Field.3
0,Dustin Johnson,Tommy Fleetwood,Bubba Watson,Branden Grace,Li Haotong,Ryan Armour,Mike Weir,Patton Kizzire,Lin Yuxin
1,Justin Thomas,Jason Day,Rafael Cabrera Bello,Louis Oosthuizen,Adam Hadwin,Daniel Berger,Bernd Wiesberger,Bernhard Langer,Doug Ghim
2,Jon Rahm,Paul Casey,Brian Harman,Patrick Cantlay,Kyle Stanley,Wesley Bryan,Danny Willett,Sandy Lyle,Russell Henley
3,Jordan Spieth,Alex Noren,Patrick Reed,Tony Finau,Cameron Smith,Angel Cabrera,Tiger Woods,Matt Parziale,Billy Horschel
4,Justin Rose,Henrik Stenson,Kevin Kisner,Ross Fisher,Thomas Pieters,Austin Cook,Ian Woosnam,Doc Redman,Larry Mize
5,Hideki Matsuyama,Marc Leishman,Xander Schauffele,Matthew Fitzpatrick,Satoshi Kodaira,Fred Couples,Yuta Ikdea,Ted Potter Jr.,Vijay Singh
6,Rory McIlroy,Tyrrell Hatton,Francesco Molinari,Daniel Berger,Dylan Frittelli,Bryson DeChambeau,Trevor Immelman,Charl Schwartzel,Jose Maria Olazabal
7,Rickie Fowler,Phil Mickelson,Charley Hoffman,Kevin Chappell,Chez Reavie,Harry Ellis,Zach Johnson,Adam Scott,Ryan Moore
8,Sergio Garcia,Pat Perez,Kiradech Aphibarnrat,Brendan Steele,Jason Dufner,Jhonattan Vegas,Martin Kaymer,Shubanhkar Sharma,Joaquin Niemann
9,Brooks Koepka,Matt Kuchar,Gary Woodland,Webb Simpson,Siwoo Kim,Jimmy Walker,Yusaku Miyazato,Webb Simpson,Mark O'Meara
